In [1]:
import pandas as pd
import numpy as np


class offline_eval_metric_cb:
    def __init__(self, group, context, action, loss, prob = None):
        """
        each parameter above is a T-dimension vector, collected from the online algorithm
        (currently in the static version...)
        group: the t-th incoming individual group, each in [m];
        context: the t-th incoming individual feature (excluding group info), each is R_{dx} vector;
        action: the t-th individual's action chosen by the agent, each in [K];
        loss: the t-th loss (one step, not cummulative up to now) obtained from that individual, each in R;
        prob: the chosen action prob (propensity score), each in Delta_{K};
        """
        self.group = np.array(group)
        self.individual_num = len(group)
        self.group_num = len(set(group))
        self.context = np.array(context)
        self.action = np.array(action)
        self.action_num = len(set(action))
        self.loss = np.array(loss)
        self.prob = np.array(prob)
        #create a dataframe including {g_t, a_t, r_t} t = 1^T
        self.offline_data = pd.DataFrame({"group": group, "action": action, "step_loss": loss})
        self.summary_loss = {}
        self.expected_action_parity_disable = True
    def group_loss_parity(self):
        cumu_group_loss = 0.001 * np.ones((self.individual_num, self.group_num))
        cumu_group_num = 0.001 * np.ones((self.individual_num, self.group_num))
        #parity between avg loss
        loss_parity = np.zeros(self.individual_num)

        #initialization
        cumu_group_loss[0][self.group[0]] = self.loss[0]
        cumu_group_num[0][self.group[0]] = 1
        loss_parity[0] = self.loss[0]
        for i in range(1, self.individual_num):
            for k in range(self.group_num):
                cumu_group_loss[i][k] = cumu_group_loss[i - 1][k]
                cumu_group_num[i][k] = cumu_group_num[i - 1][k]

            cumu_group_loss[i][self.group[i]] += self.loss[i]
            cumu_group_num[i][self.group[i]] += 1
            #compute the loss parity at each time t
            avg_loss = [cumu_group_loss[i][k] / cumu_group_num[i][k] for k in range(self.group_num)]
            avg_loss.sort()
            loss_parity[i] = avg_loss[-1] - avg_loss[0]
        cumu_group_loss = cumu_group_loss.T
        cumu_group_num = cumu_group_num.T
        cumu_loss = np.sum(cumu_group_loss, axis = 0)
        
        self.offline_data['cumu_loss'] = cumu_loss
        self.summary_loss = {'cumu_loss': cumu_loss[-1]}
        for k in range(self.group_num):
            self.offline_data['cumu_loss_' + str(k)] = list(cumu_group_loss[k])
            self.summary_loss['cumu_loss_' + str(k)] = cumu_group_loss[k][-1]
            self.offline_data['cumu_num_' + str(k)] = list(cumu_group_num[k])
        self.offline_data['loss_parity'] = list(loss_parity)
        self.summary_loss['loss_parity'] = loss_parity[-1]
        return self.summary_loss

    
    def group_action_parity_computation(self, prefix = 'realized'):
        """
        realized action parity: from the realized action; 
        expected action parity: from the expected action simplex.
        """
        ## 0.001 prevents the case where 0 is the denominator
        avg_action_num = [0.001] * self.action_num
        group_action_num = 0.001 * np.ones((self.group_num, self.action_num))
        ## initialization
        avg_action_num[self.action[0]] += 1
        group_action_num[self.group[0]][self.action[0]] += 1
        max_group_action_parity = np.zeros(self.individual_num)
        for i in range(1, self.individual_num):
            ## update num
            if prefix == 'realized':
                avg_action_num[self.action[i]] += 1
                group_action_num[self.group[i]][self.action[i]] += 1
            elif prefix == 'expected':
                #see from the prob table
                for j in range(self.action_num):
                    avg_action_num[j] += self.prob[i][j]
                    group_action_num[self.group[i]][j] += self.prob[i][j]
            else:
                raise NotImplementedError
            ## update parity directly
            group_action_parity = np.zeros(self.group_num)
            for k in range(self.group_num):
                group_action_parity[k] = np.sum([abs(avg_action_num[j]/(i + 1) - group_action_num[k][j]/sum(group_action_num[k])) 
                                                for j in range(self.action_num)])
            print(group_action_parity)
            max_group_action_parity[i] = max(group_action_parity)
        #print(avg_action_num, group_action_num)
        self.offline_data['group_action_parity_' + prefix] = list(max_group_action_parity)
        self.summary_loss['group_action_parity_' + prefix] = max_group_action_parity[-1]
        return self.summary_loss

    def group_action_parity(self):
        self.group_action_parity_computation('realized')
        if self.expected_action_parity_disable == False:
            self.group_action_parity_computation('expected')
        
    def individual_loss_parity_computation(self, prefix = 'realized', time_window = None, order = None):
        """
        order: one-side or two-side (i.e. full time or hindsight), default None means two-side, i.e. the action for one individual is fair to both former and latter.
        time_window: default None mean compute the unfairness from the start time until now, otherwise only compute K-step unfairness (moving window)
        action_distance encoding (realized): if action are different, label action_distance to be 1 otherwise 0.
        =sum indicate function {d(a_1, a_2) >= c1 * d(x1, x_2) + c2}
        """
        loss_parity = np.zeros(self.individual_num) 
        c1 = 0.1
        c2 = 0.5
        if time_window == None:
            if order == None:
                for t1 in range(1, self.individual_num):
                    loss_parity[t1] = loss_parity[t1 - 1]
                    for t2 in range(t1):
                        if self.action[t1] != self.action[t2]:
                            #compute the feature distance to see if there is some unfairness
                            if c1 * np.linalg.norm(self.context[t1] - self.context[t2]) + c2 >= 1:
                                loss_parity[t1] += 1
                self.offline_data['individual_loss_parity_' + prefix] = list(loss_parity)
                self.summary_loss['individual_loss_parity_' + prefix] = loss_parity[-1]
            else:
                raise NotImplementedError


       
        else:
            #(TODO): add other versions
            raise NotImplementedError    

    def individual_loss_parity(self):
        #involve the interaction with the context...
        self.individual_loss_parity_computation('realized')





In [2]:
import pandas as pd
import numpy as np
group = [0, 1, 0, 0, 1]
context = [3, 4, 2, 6, 1]
action = [1, 0, 1, 0, 1]
loss = [0.4, 0.2, 0.6, 0.1, 0.5]
prob = [[0.2, 0.8], [0.6, 0.4], [0.1, 0.9], [0.7, 0.3], [0.2, 0.8]]
metrics = offline_eval_metric_cb(group, context, action, loss, prob)
metrics.group_loss_parity()
metrics.group_action_parity()
metrics.individual_loss_parity()
metrics.offline_data


[0.99800399 0.99800399]
[0.66566767 1.33133733]
[0.33311126 0.99800399]
[0.13311126 0.2       ]


,group,action,step_loss,cumu_loss,cumu_loss_0,cumu_num_0,cumu_loss_1,cumu_num_1,loss_parity,group_action_parity_realized,individual_loss_parity_realized
0,0,1,0.4,0.401,0.4,1.0,0.001,0.001,0.400000,0.000000,0.0
1,1,0,0.2,0.601,0.4,1.0,0.201,1.001,0.199201,0.998004,0.0
2,0,1,0.6,1.201,1.0,2.0,0.201,1.001,0.299201,1.331337,0.0
3,0,0,0.1,1.301,1.1,3.0,0.201,1.001,0.165867,0.998004,0.0
4,1,1,0.5,1.801,1.1,3.0,0.701,2.001,0.016342,0.200000,1.0


In [45]:
metrics.summary_loss

{'cumu_loss': 1.8,
 'cumu_loss_0': 1.1,
 'cumu_loss_1': 0.7,
 'loss_parity': 0.01666666666666672,
 'group_action_parity_realized': 0.19999999999999996,
 'group_action_parity_expected': 0.15999999999999992}

In [ ]:
## the falcon procedure, for the paper
## https://arxiv.org/pdf/2003.12699.pdf
from src.models.cb.FALCON import FALCON, FALCON_ldf, FALCON_price
#from split_gz_csv import ds_files_csv, ds_files
#from split import sample_custom_pmf
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
#from glob_param import *
from src.run_vw_job import process
#from split import *
import re
rgx = re.compile('^average loss = (.*)$', flags=re.M)
#show the regret performance in each algorithm

csv_path = 'src/datasets/adult/adult_2.csv'
# def alg_performance_regret(DS_DIR, did, alg_param, did_type):
loss_param = []
    # if did_type == 'openml':
    #     sample_falcon = FALCON(csv_path, 95, 1)
    # elif did_type == 'ms':
    #     sample_falcon = FALCON_ldf(csv_path, 105, 1, 10)
    # elif did_type == 'yahoo':
    #     sample_falcon = FALCON_ldf(csv_path, 105, 1, 6, 'ridge')
    # elif did_type == 'loan':
    #     if did == 0:
    #         sample_falcon = FALCON_price(csv_path, 10, 1, 15)
    #     elif did == 1:
    #         sample_falcon = FALCON_price(csv_path, 10, 1, 10)
#gamma_param is a hyperparam in FALCON alg.
sample_falcon = FALCON(csvpath = csv_path, gamma_param = 95, feed_choice = 1,
                        group = ['sex', 'race'])
    
sample_falcon.learn_schedule()

#cummulative loss for FALCON alg.
sample_falcon.loss_all
        
    # gz_path = ds_files(DS_DIR)[did]
    # for index in range(len(alg_param)):
    #     pv_loss, output = process(gz_path, alg_param[index], None, True, did_type)
    #     loss_param.append(list(output_extraction(output)))
    # return loss_param

3


array([0., 0., 0., ..., 0., 0., 0.])